**Installations [Hidden cell]**

In [1]:
import sys
sys.path.append('/media/nicholasjprimiano/kerasapplications/')
sys.path.append('/media/nicholasjprimiano/efficientnet-keras-source-code/')
import keras_applications
import efficientnet.tfkeras as efficientnet

**Imports [Hidden cell]**

# RSNA - EfficientNet Baseline [TF]
_____

### EfficientNet

EfficientNet is one of the most solid baselines known today. It is a family of convolutional neural networks that have achieved state-of-the-art accuracy on ImageNet while also being smaller and faster than other models.
The main idea of EfficientNet is scaling up CNNs in a principled way. It uses a scalable architecture, named compound scaling, which balances network depth, width, and resolution to achieve superior performance.
The image below shows the scaling method in more detail.

![](https://i.ibb.co/Y86KGDg/image4-1.png)

#### Model Size (B5)

As written above, the main claim of efficientnet is providing a method for scaling up neural networks. 
Using this approach, the authors of the paper released the official efficientnet architecture scaled to various sizes [B0, B1, B2.. B7. And two monstrosities L1 and L2].
As it is usually empirically the case (Also hinted by the [Scaling Laws for Neural Language Models](https://arxiv.org/abs/2001.08361)): Performance increase with model size. 
On this notebook we use the **B5** variant of efficientnet. 
Feel free to experiment with larger sizes.

In [2]:
import os
import cv2
import glob
import traceback
import cv2 as cv
import numpy as np
import pandas as pd
from path import Path
from tqdm import tqdm
import nibabel as nib
import pydicom as dicom
import tensorflow as tf
from keras import layers
from pydicom import dcmread
from tensorflow import keras
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from pydicom.data import get_testdata_files
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D
from tensorflow.keras.preprocessing.image import load_img, img_to_array

### Data Loading

> **Note:** Mean target calculation for a baseline submission.

In [3]:
bad = np.array([['1.2.826.0.1.3680043.10197_C1', '1.2.826.0.1.3680043.10197','C1'],['1.2.826.0.1.3680043.10454_C1', '1.2.826.0.1.3680043.10454','C1'],['1.2.826.0.1.3680043.10690_C1', '1.2.826.0.1.3680043.10690','C1']], dtype=object)

train_df = pd.read_csv("/media/nicholasjprimiano/rsna-2022-cervical-spine-fracture-detection/train.csv")
test_df = pd.read_csv("/media/nicholasjprimiano/rsna-2022-cervical-spine-fracture-detection/test.csv")

train_dir = '/media/nicholasjprimiano/rsna-2022-cervical-spine-fracture-detection/train_images'
test_dir = '/media/nicholasjprimiano/rsna-2022-cervical-spine-fracture-detection/test_images'
first_image = os.path.join(test_dir, test_df['StudyInstanceUID'].iloc[0])

new_submission = []
means = train_df.median(numeric_only=True).to_dict()
means = dict(zip(train_df.columns[1:], np.average(train_df[train_df.columns[1:]], axis=0, weights=train_df["patient_overall"] + 1)))
prediction_type = test_df['prediction_type'].tolist()
submission = pd.read_csv('/media/nicholasjprimiano/rsna-2022-cervical-spine-fracture-detection/sample_submission.csv')
for i in range(len(submission)):        
    new_submission.append(means[prediction_type[i]])
submission['fractured'] = new_submission


if(test_df.values[0][0] == bad[0][0]): test_df = pd.DataFrame({"row_id": ['1.2.826.0.1.3680043.22327_C1', '1.2.826.0.1.3680043.25399_C1', '1.2.826.0.1.3680043.5876_C1'], "StudyInstanceUID": ['1.2.826.0.1.3680043.22327', '1.2.826.0.1.3680043.25399', '1.2.826.0.1.3680043.5876'], "prediction_type": ["C1", "C1", "C1"]})  
prediction_type_mapping = test_df['prediction_type'].map({'C1': 0, 'C2': 1, 'C3': 2, 'C4': 3, 'C5': 4, 'C6': 5, 'C7': 6}).values

/tmp/ipykernel_15427/167522624.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  bad = np.array([['1.2.826.0.1.3680043.10197_C1', '1.2.826.0.1.3680043.10197','C1'],['1.2.826.0.1.3680043.10454_C1', '1.2.826.0.1.3680043.10454','C1'],['1.2.826.0.1.3680043.10690_C1', '1.2.826.0.1.3680043.10690','C1']], dtype=np.object)


#### Utility Functions & Variables

- **load_dicom:** For loading (and converting) a single dicom image.
- **lisdirs:** For listing the data directory.
- **train_dir / test_dir:** used later on for fast switching between train & validation paths

In [4]:
def load_dicom(path, size = 64):
    try:
        img=dicom.dcmread(path)
        img.PhotometricInterpretation = 'YBR_FULL'
        data=img.pixel_array
        data=data-np.min(data)
        if np.max(data) != 0:
            data=data/np.max(data)
        data=(data*255).astype(np.uint8)        
        return cv2.cvtColor(data.reshape(512, 512), cv2.COLOR_GRAY2RGB)
    except:        
        return np.zeros((512, 512, 3))

def listdirs(folder):
    return [d for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]    

train_dir = '/media/nicholasjprimiano/rsna-2022-cervical-spine-fracture-detection/train_images'
test_dir = '/media/nicholasjprimiano/rsna-2022-cervical-spine-fracture-detection/test_images'
patients = sorted(os.listdir(train_dir))

#### Quick Visualizations

> **Credit:** This great [notebook](https://www.kaggle.com/code/realneuralnetwork/rsna-efficientnet-infer) by [Kabir Ivan](https://www.kaggle.com/realneuralnetwork)

Simple sanity test - mainly for testing out the load_dicom function.load_dicom

**Training Samples**

In [5]:
"""image_file = glob.glob("/media/nicholasjprimiano/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.10001/*.dcm")
plt.figure(figsize=(20, 20))

for i in range(28):
    ax = plt.subplot(7, 7, i + 1)
    image_path = image_file[i]
    image = load_dicom(image_path)
    plt.axis('off')   
    plt.imshow(image)"""

'image_file = glob.glob("/media/nicholasjprimiano/rsna-2022-cervical-spine-fracture-detection/train_images/1.2.826.0.1.3680043.10001/*.dcm")\nplt.figure(figsize=(20, 20))\n\nfor i in range(28):\n    ax = plt.subplot(7, 7, i + 1)\n    image_path = image_file[i]\n    image = load_dicom(image_path)\n    plt.axis(\'off\')   \n    plt.imshow(image)'

**Targets**

In [6]:
"""image_file = glob.glob("/media/nicholasjprimiano/rsna-2022-cervical-spine-fracture-detection/segmentations/*.nii")
plt.figure(figsize=(20, 20))

for i in range(28):
    ax = plt.subplot(7, 7, i + 1)
    image_path = image_file[i]
    nii_img = nib.load(image_path).get_fdata()
    nib_image = nii_img[:,:,59]
    plt.axis('off')
    plt.imshow(nib_image)"""

'image_file = glob.glob("/media/nicholasjprimiano/rsna-2022-cervical-spine-fracture-detection/segmentations/*.nii")\nplt.figure(figsize=(20, 20))\n\nfor i in range(28):\n    ax = plt.subplot(7, 7, i + 1)\n    image_path = image_file[i]\n    nii_img = nib.load(image_path).get_fdata()\n    nib_image = nii_img[:,:,59]\n    plt.axis(\'off\')\n    plt.imshow(nib_image)'

### Data Generators

Since the training data can be a bit large, we load it batch by batch when training & inference.
This is done through simple keras generators. 

#### Train Generator 

- Also yields labels

In [7]:
def RSNATrainGenerator(train_df, batch_size=64, infinite = True, base_path = train_dir):
    while True:
        trainset = []
        trainidt = []
        trainlabel = []
        for i in (range(len(train_df))):
            idt = train_df.loc[i, 'StudyInstanceUID']
            path = os.path.join(train_dir, idt)
            for im in os.listdir(path):
                dc = dicom.read_file(os.path.join(path,im))
                if dc.file_meta.TransferSyntaxUID.name =='JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])':
                    continue
                img = load_dicom(os.path.join(path , im))
                img = cv.resize(img, (128 , 128))
                image = img_to_array(img)
                image = image / 255.0
                trainset += [image]
                cur_label = []
                cur_label.append(train_df.loc[i,'C1'])
                cur_label.append(train_df.loc[i,'C2'])
                cur_label.append(train_df.loc[i,'C3'])
                cur_label.append(train_df.loc[i,'C4'])
                cur_label.append(train_df.loc[i,'C5'])
                cur_label.append(train_df.loc[i,'C6'])
                cur_label.append(train_df.loc[i,'C7'])
                trainlabel += [cur_label]
                trainidt += [idt]
                if len(trainidt) == batch_size:                    
                    yield np.array(trainset), np.array(trainlabel)
                    trainset, trainlabel, trainidt = [], [], []
            i+=1

In [8]:
"""
img, label = next(RSNATrainGenerator(train_df, batch_size=12, infinite = True, base_path = train_dir))
plt.imshow(img[0,:,:,:])
print(img.shape)
print(label.shape)"""

'\nimg, label = next(RSNATrainGenerator(train_df, batch_size=12, infinite = True, base_path = train_dir))\nplt.imshow(img[0,:,:,:])\nprint(img.shape)\nprint(label.shape)'

In [9]:
"""count = 0
batch_size = 2
return_vals = ()
while count < 2:
    trainset = []
    trainidt = []
    trainlabel = []
    for i in (range(len(train_df))):
        idt = train_df.loc[i, 'StudyInstanceUID']
        path = os.path.join(train_dir, idt)
        for im in os.listdir(path):
            dc = dicom.read_file(os.path.join(path,im))
            if dc.file_meta.TransferSyntaxUID.name =='JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])':
                continue
            img = load_dicom(os.path.join(path , im))
            img = cv.resize(img, (128 , 128))
            image = img_to_array(img)
            plt.imshow(image)
            image = image / 255.0
            trainset += [image]
            cur_label = []
            cur_label.append(train_df.loc[i,'C1'])
            cur_label.append(train_df.loc[i,'C2'])
            cur_label.append(train_df.loc[i,'C3'])
            cur_label.append(train_df.loc[i,'C4'])
            cur_label.append(train_df.loc[i,'C5'])
            cur_label.append(train_df.loc[i,'C6'])
            cur_label.append(train_df.loc[i,'C7'])
            trainlabel += [cur_label]
            trainidt += [idt]
            if len(trainidt) == batch_size:                    
                return_vals = np.array(trainset), np.array(trainlabel)
                trainset, trainlabel, trainidt = [], [], []
        i+=1
        count+=1"""

"count = 0\nbatch_size = 2\nreturn_vals = ()\nwhile count < 2:\n    trainset = []\n    trainidt = []\n    trainlabel = []\n    for i in (range(len(train_df))):\n        idt = train_df.loc[i, 'StudyInstanceUID']\n        path = os.path.join(train_dir, idt)\n        for im in os.listdir(path):\n            dc = dicom.read_file(os.path.join(path,im))\n            if dc.file_meta.TransferSyntaxUID.name =='JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])':\n                continue\n            img = load_dicom(os.path.join(path , im))\n            img = cv.resize(img, (128 , 128))\n            image = img_to_array(img)\n            plt.imshow(image)\n            image = image / 255.0\n            trainset += [image]\n            cur_label = []\n            cur_label.append(train_df.loc[i,'C1'])\n            cur_label.append(train_df.loc[i,'C2'])\n            cur_label.append(train_df.loc[i,'C3'])\n            cur_label.append(train_df.loc[i,'C4'])\n 

In [10]:
"""path = os.path.join(train_dir, idt)

im = os.listdir(path)

#print(im)

dc = dicom.read_file(os.path.join(path,im[10]))

#print(dc.file_meta.TransferSyntaxUID.name)

img = load_dicom(os.path.join(path , im[10]))

img = cv.resize(img, (128 , 128))

image = img_to_array(img)

plt.imshow(image)
image = image / 255.0

plt.imshow(image)

print(len([image]))"""

'path = os.path.join(train_dir, idt)\n\nim = os.listdir(path)\n\n#print(im)\n\ndc = dicom.read_file(os.path.join(path,im[10]))\n\n#print(dc.file_meta.TransferSyntaxUID.name)\n\nimg = load_dicom(os.path.join(path , im[10]))\n\nimg = cv.resize(img, (128 , 128))\n\nimage = img_to_array(img)\n\nplt.imshow(image)\nimage = image / 255.0\n\nplt.imshow(image)\n\nprint(len([image]))'

#### Test Generator

- Yields only image samples

In [11]:
def RSNATestGenerator(test_df, batch_size, infinite = True, base_path = test_dir):
    while 1:        
        testset=[]
        testidt=[]
        for i in (range(len(test_df))):        
            if type(test_df) is list: idt = test_df[i]
            else: idt = test_df['StudyInstanceUID'].iloc[i]
            path = os.path.join(base_path, idt)
            if os.path.exists(path):
                for im in os.listdir(path):
                    dc = dicom.read_file(os.path.join(path,im))
                    if dc.file_meta.TransferSyntaxUID.name =='JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])':
                        continue
                    img=load_dicom(os.path.join(path,im))
                    img=cv.resize(img,(128, 128))
                    image=img_to_array(img)
                    image=image/255.0
                    testset+=[image]
                    testidt+=[idt]
                    if len(testset) == batch_size:                        
                        yield np.array(testset)
                        testset = []
        if len(testset) > 0: yield np.array(testset)
        if not infinite: break

#### The Model

> **Note:** The training data loads as a single channel image, we use a simple `Conv2D` to align it to be a 3 channel image (As expected by efficientnet).

In [30]:
from tensorflow.keras.applications import ResNet152V2

def get_model():
    inp = keras.layers.Input((None, None ,1))
    x = Conv2D(3, 3, padding = 'SAME')(inp)
    #x = efficientnet.EfficientNetB5(include_top=False, weights='/media/nicholasjprimiano/efficientnet-b5_advprop_notop.h5')(x)
    x = ResNet152V2(include_top=False)(x)
    x = keras.layers.GlobalAveragePooling2D()(x)
    out = keras.layers.Dense(7, 'sigmoid')(x)
    model = keras.models.Model(inp, out)
    model.summary()
    model.compile(loss="binary_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001), metrics = ['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    return model

model = get_model()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, None, 1)]   0         
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 3)     30        
                                                                 
 resnet152v2 (Functional)    (None, None, None, 2048)  58331648  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 7)                 14343     
                                                                 
Total params: 58,346,021
Trainable params: 58,202,277
Non-trainable params: 143,744
___________________________________________

In [14]:
"""def get_model(width=128, height=128, depth=64):
    #Build a 3D convolutional neural network model.

    inputs = keras.layers.Input((128, 128, 64, 1))

    x = layers.Conv3D(filters=64, kernel_size=2, activation="relu")(inputs)
    #x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    #x = layers.Conv3D(filters=64, kernel_size=2, activation="relu")(x)
    #x = layers.MaxPool3D(pool_size=2)(x)
    #x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=2, activation="relu")(x)
    #x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    
    x = layers.Conv3D(filters=256, kernel_size=1, activation="relu")(x)
    #x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    out = keras.layers.Dense(7, 'sigmoid')(x)
    model = keras.models.Model(inputs, out)
    return model

# Build model.
model = get_model(width=128, height=128, depth=64)
model.summary()

# Compile model.
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True)

model.compile(
loss="binary_crossentropy",
optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
metrics=["acc"],)"""
"""tf.keras.utils.plot_model(model, show_shapes=True)"""

'def get_model(width=128, height=128, depth=64):\n    #Build a 3D convolutional neural network model.\n\n    inputs = keras.layers.Input((128, 128, 64, 1))\n\n    x = layers.Conv3D(filters=64, kernel_size=2, activation="relu")(inputs)\n    #x = layers.MaxPool3D(pool_size=2)(x)\n    x = layers.BatchNormalization()(x)\n\n    #x = layers.Conv3D(filters=64, kernel_size=2, activation="relu")(x)\n    #x = layers.MaxPool3D(pool_size=2)(x)\n    #x = layers.BatchNormalization()(x)\n\n    x = layers.Conv3D(filters=128, kernel_size=2, activation="relu")(x)\n    #x = layers.MaxPool3D(pool_size=2)(x)\n    x = layers.BatchNormalization()(x)\n    \n    x = layers.Conv3D(filters=256, kernel_size=1, activation="relu")(x)\n    #x = layers.MaxPool3D(pool_size=2)(x)\n    x = layers.BatchNormalization()(x)\n\n    x = layers.GlobalAveragePooling3D()(x)\n    x = layers.Dense(units=512, activation="relu")(x)\n    x = layers.Dropout(0.3)(x)\n\n    out = keras.layers.Dense(7, \'sigmoid\')(x)\n    model = keras.

#### Main Training Cell

We use a stratified (based on patient_overall) KFold split and train 5 different models. 

In [31]:
for train_idx, val_idx in StratifiedKFold(5).split(train_df, train_df['patient_overall']):    
    K.clear_session()
    x_train = train_df.iloc[train_idx].reset_index()
    x_val = train_df.iloc[val_idx].reset_index()

    hist = model.fit(                            
                                    RSNATrainGenerator(x_train, min(len(x_train), 64), infinite = False, base_path = train_dir),
                                    epochs = 50,
                                    verbose = 1,
                                    callbacks = [keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 2, restore_best_weights = True)],
                                    validation_steps = max((len(x_val) // 64), 1),
                                    steps_per_epoch = max((len(x_train) // 64), 1),
                                    validation_data = RSNATrainGenerator(x_val, min(len(x_val), 64), infinite = False, base_path = train_dir),
                              )


Epoch 1/50
25/25 [==============================] - 14s 272ms/step - loss: 0.1904 - accuracy: 0.1169 - auc: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 2.7618 - val_accuracy: 0.0000e+00 - val_auc: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
25/25 [==============================] - 6s 227ms/step - loss: 0.1355 - accuracy: 0.1912 - auc: 0.6770 - precision: 0.0000e+00 - recall: 0.0000e+00 - val_loss: 2.6878 - val_accuracy: 0.0000e+00 - val_auc: 0.4978 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/50
25/25 [==============================] - 5s 210ms/step - loss: 0.3635 - accuracy: 0.0994 - auc: 0.7937 - precision: 0.4586 - recall: 0.3600 - val_loss: 4.5515 - val_accuracy: 0.0000e+00 - val_auc: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/50
25/25 [==============================] - 5s 214ms/step - loss: 0.2617 - accuracy: 0.2000 - auc: 0.8002 - precision: 0.6078 - recall: 0.2868 - val_loss: 0.9078 - val

2022-09-11 09:34:04.862170: W tensorflow/core/framework/op_kernel.cc:1733] INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_74 in the registry.
Traceback (most recent call last):

  File "/home/nicholasjprimiano/.local/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 263, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc_74 in the registry.


2022-09-11 09:34:04.862220: W tensorflow/core/kernels/data/generator_dataset_op.cc:107] Error occurred when finalizing GeneratorDataset iterator: INVALID_ARGUMENT: ValueError: Could not find callback with key=pyfunc_74 in the registry.
Traceback (most recent call last):

  File "/home/nicholasjprimiano/.local/lib/python3.9/site-packages/tensorflow/python/ops/script_ops.py", line 263, in __call__
    raise ValueError(f"Could not find callback with key={token} in the "

ValueError: Could not find callback with key=pyfunc

Epoch 1/50
25/25 [==============================] - 9s 259ms/step - loss: 0.4112 - accuracy: 0.3594 - auc: 0.7866 - precision: 0.5115 - recall: 0.3158 - val_loss: 0.0599 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/50
25/25 [==============================] - 5s 209ms/step - loss: 0.2463 - accuracy: 0.3181 - auc: 0.8364 - precision: 0.4296 - recall: 0.3452 - val_loss: 0.3634 - val_accuracy: 0.0000e+00 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/50
25/25 [==============================] - 5s 214ms/step - loss: 0.2877 - accuracy: 0.2812 - auc: 0.7872 - precision: 0.4288 - recall: 0.3096 - val_loss: 0.1367 - val_accuracy: 0.2318 - val_auc: 0.0000e+00 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 1/50
25/25 [==============================] - 9s 253ms/step - loss: 0.4692 - accuracy: 0.3963 - auc: 0.7437 - precision: 0.3644 - recall: 0.2384 - val_loss: 0.0857 - val_accuracy: 0.

In [17]:
val_pred = model.predict(RSNATestGenerator(x_val, min(len(test_df), 64), infinite = False, base_path = train_dir), steps = max((len(test_df) // 64), 1))    
try: # the best we can do at the moment..
    preds = model.predict(RSNATestGenerator(test_df, min(len(test_df), 64), infinite = False, base_path = test_dir), steps = max((len(test_df) // 64), 1))
    new_preds = []
    for pred_idx in range(len(preds)):
        new_preds.append(preds[pred_idx][prediction_type_mapping[pred_idx]])
    # submission['fractured'] += preds[:, prediction_type_mapping] / 5
    submission['fractured'] += np.array(new_preds) / 5      
except: traceback.print_exc()   

#### Submission

In [18]:
"""submission
submission.to_csv('submission.csv', index = 0)"""

"submission\nsubmission.to_csv('submission.csv', index = 0)"